In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# load the data
def load_data(file):
    return pd.read_csv(file)

# compute summary statistics of data
def summarise_data(df):
    print(df.describe())

# call functions
df = load_data('London_Listings.csv')
summarise_data(df)
print(df.dtypes)

In [ ]:
# visualise missing values
def visualize_missing_values(df, threshold=0.0):
    # find percentage of missing vals
    missing_percent = (df.isnull().sum() / len(df)) * 100

    # filter columns above threshold
    missing_percent = missing_percent[missing_percent > threshold]

    if missing_percent.empty:
        print("No missing values above threshold found in the dataset.")
        return

    # sort descending
    missing_percent = missing_percent.sort_values(ascending=False)

    # plot bar chart
    plt.figure(figsize=(12, 6))
    ax = missing_percent.plot(kind='bar', color='skyblue', edgecolor='black')

    # annotate bars with %
    for i, v in enumerate(missing_percent):
        ax.text(i, v + 0.5, f"{v:.1f}%", ha='center', va='bottom', fontsize=8, rotation=90)

    plt.title('Missing Values per Column (%)')
    plt.xlabel('Columns')
    plt.ylabel('Percentage Missing')
    plt.xticks(rotation=90, ha='right')
    plt.tight_layout()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

visualize_missing_values(df)

In [ ]:
# inital basic data cleaning
def initial_clean(df):
    # replace N/A with np.nan
    df.replace("N/A", np.nan, inplace=True)
    
    # convert percentages
    df['host_response_rate'] = df['host_response_rate'].str.rstrip('%').astype(float)
    df['host_acceptance_rate'] = df['host_acceptance_rate'].str.rstrip('%').astype(float)
    
    # remove dollar sign and comma from price
    df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)

    # convert to bool
    df['host_is_superhost'] = df['host_is_superhost'].map({'t': 1, 'f': 0})

    # date parsing
    date_cols = ['host_since', 'calendar_last_scraped', 'first_review', 'last_review']
    for col in date_cols:
        df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)


    return df

# dropping irrelvant cols & those with low predictive power
def remove_cols(df):
    cols_to_drop = [
        'id',
        'name',
        'description',
        'host_id',
        'host_name',
        'host_listings_count',
        'calculated_host_listings_count',
        'calendar_last_scraped'
    ]
    df = df.drop(columns=[col for col in cols_to_drop if col in df.columns])
    return df

df = remove_cols(initial_clean(df))
summarise_data(df)

In [ ]:
# handle missing values - prevent skewing
def handle_missing(df):
    df = df.copy()
    # remove rows with missing price
    df = df[~df['price'].isnull()]

    # replace review_scores_rating with NaN if there are no reviews
    df['has_reviews'] = (df['number_of_reviews'] > 0).astype(int)
    df.loc[df['number_of_reviews'] == 0, 'review_scores_rating'] = np.nan

    # fill with grouped median of similar properties
    df['review_scores_rating'] = df['review_scores_rating'].fillna(
        df.groupby(['neighbourhood', 'room_type', 'accommodates'])['review_scores_rating'].transform('median')
    )

    # time since last reviews
    today = pd.Timestamp('today')
    df['days_since_first_review'] = (today - df['first_review']).dt.days.fillna(0)
    df['days_since_last_review'] = (today - df['last_review']).dt.days.fillna(0)
    df.drop(columns=['first_review', 'last_review'], inplace=True)

    # flag missing response rates and fill with median
    df['response_rate_missing'] = df['host_response_rate'].isnull().astype(int)
    df['host_response_rate'] = df['host_response_rate'].fillna(df['host_response_rate'].median())

    # flag missing acceptance rates and fill with median
    df['acceptance_rate_missing'] = df['host_acceptance_rate'].isnull().astype(int)
    df['host_acceptance_rate'] = df['host_acceptance_rate'].fillna(df['host_acceptance_rate'].median())

    # fill with median for similar properties
    df['beds'] = df.groupby(['accommodates', 'room_type'])['beds'].transform(
        lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x.fillna(x.median())
    )
    df['bedrooms'] = df.groupby(['accommodates', 'room_type'])['bedrooms'].transform(
        lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x.fillna(x.median())
    )

    # fill with medians
    df['bathrooms'] = df['bathrooms'].fillna(df['bathrooms'].median())
    df['host_is_superhost'] = df['host_is_superhost'].fillna(df['host_is_superhost'].mode()[0])
    df['host_total_listings_count'] =  df['host_total_listings_count'].fillna(df['host_total_listings_count'].median())
    
    # fill with median
    median_date = df['host_since'].dropna().median()
    df['host_since'] = df['host_since'].fillna(median_date)
    
    return df

df = handle_missing(df)


In [ ]:
# log transform price
def transform_price(df):
    df['log_price'] = np.log1p(df['price'])

# log transform listings count
def transform_listings_count(df):
    df['log_host_listings'] = np.log1p(df['host_total_listings_count'])

# winsorise outliers
def handle_outliers(df, columns, lower_q=0.01, upper_q=0.99):
    df = df.copy()
    for col in columns:
        if col in df.columns:
            lower = df[col].quantile(lower_q)
            upper = df[col].quantile(upper_q)
            df[col] = df[col].clip(lower, upper)
    return df

transform_price(df)
transform_listings_count(df)
handle_outliers(df, ['beds','accommodates','bedrooms'])